# Eigenvalue problems (Unit 2.2) 

We solve the generalized eigenvalue problem
$$A u=\lambda M u$$,

where A comes from $\int \nabla u \nabla v$, and $M$ from $\int uv$, on the space $H_0^1$

In [1]:
import netgen.gui
%gui tk
from netgen.geom2d import unit_square
from ngsolve import *
from math import pi
import scipy.linalg
from scipy import random

mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))


We setup a stiffness matrix A and a mass matrix M, and declare a preconditioner for A:

In [2]:
fes = H1(mesh, order=4, dirichlet=[1,2,3,4])
u = fes.TrialFunction()
v = fes.TestFunction()

a = BilinearForm(fes)
a += SymbolicBFI(grad(u)*grad(v))
pre = Preconditioner(a, "multigrid")

m = BilinearForm(fes)
m += SymbolicBFI(u*v)

a.Assemble()
m.Assemble()

u = GridFunction(fes)

The inverse iteration is
$$u_{n+1}=A^{−1}Mu_n,$$

where the Rayleigh quotient
$$\rho_n=\frac{\langle Au_n,u_n \rangle}{\langle Mu_n,u_n\rangle}$$

converges to the smallest eigen-value, with rate of convergence $\lambda_1/\lambda_2$.

The preconditioned inverse iteration (PINVIT), see [Knyazef+Neymeyr], replaces $A^{−1}$
by an approximate inverse $C^{−1}$

$$\rho_n=\frac{\langle Au_n,u_n\rangle}{\langle Mu_n,u_n\rangle}$$

$$w_n=C^{−1}(Au_n−\rho Mu_n)$$
$$u_{n+1}=u_n+\alpha w_n$$

The optimal step-size $\alpha$ is found by minimizing the Rayleigh-quotient on a two-dimensional space:
$$u_{n+1}=\arg \min_{v\in \lbrace u_n,w_n\rbrace} \frac{\langle Av,v\rangle}{\langle Mv,v\rangle}$$

This minimization problem can be solved by a small eigenvalue problem
$$ay=\lambda my$$

with matrices
$$a = \left(
\begin{array}[cc]
\ \langle Au_n,u_n\rangle & \langle Au_n,w_n\rangle\\
\langle Aw_n,u_n\rangle & \langle Aw_n,w_n\rangle
\end{array}
\right) \hspace{3em} m = \left(
\begin{array}[cc]
\ \langle Mu_n,u_n\rangle & \langle Mu_n,w_n\rangle\\
\langle Mw_n,u_n\rangle & \langle Mw_n,w_n\rangle
\end{array}
\right) $$

Then, the new iterate is

$$u_{n+1}=y_1 u_n+y_2 w_n$$

where $y$ is the eigenvector corresponding to the smaller eigenvalue.
### Implementation in NGSolve

Create some help vectors:


In [3]:
r = u.vec.CreateVector()
w = u.vec.CreateVector()
Mu = u.vec.CreateVector()
Au = u.vec.CreateVector()
Mw = u.vec.CreateVector()
Aw = u.vec.CreateVector()


random initial condition, zero on the boundary

In [4]:
# Note how we can use numpy with NGSolve vectors!
r.FV().NumPy()[:] = random.rand(fes.ndof)
# Linear operator projecting to true/false bits of BitArray
# mask, depending on argument range
# Also has a Project() method to project a vector inline
u.vec.data = Projector(fes.FreeDofs(), range=True) * r

# old way...
#freedofs = fes.FreeDofs()    
#for i in range(len(u.vec)):
#    u.vec[i] = random() if freedofs[i] else 0   

In [5]:
for i in range(20):
    Au.data = a.mat * u.vec
    Mu.data = m.mat * u.vec
    auu = InnerProduct(Au, u.vec)
    muu = InnerProduct(Mu, u.vec)
    # Rayleigh quotiont
    lam = auu/muu
    print (lam / (pi**2))
    # residual
    r.data = Au - lam * Mu
    w.data = pre.mat * r.data
    w.data = 1/Norm(w) * w
    Aw.data = a.mat * w
    Mw.data = m.mat * w

    # setup and solve 2x2 small eigenvalue problem
    asmall = Matrix(2,2)
    asmall[0,0] = auu
    asmall[0,1] = asmall[1,0] = InnerProduct(Au, w)
    asmall[1,1] = InnerProduct(Aw, w)
    msmall = Matrix(2,2)
    msmall[0,0] = muu
    msmall[0,1] = msmall[1,0] = InnerProduct(Mu, w)
    msmall[1,1] = InnerProduct(Mw, w)
    # print ("asmall =", asmall, ", msmall = ", msmall)
    
    
    eval,evec = scipy.linalg.eigh(a=asmall, b=msmall)
    # print (eval, evec)
    u.vec.data = float(evec[0,0]) * u.vec + float(evec[1,0]) * w
    
Draw (u)

26.83126293693843
2.0301813451116
2.0009740208150095
2.000161421001047
2.00003814124705
2.000009616822793
2.000002482567583
2.0000006505790657
2.0000001722727845
2.0000000459906015
2.000000012371347
2.000000003365789
2.000000000941486
2.000000000286302
2.000000000108649
2.0000000000603486
2.0000000000471863
2.0000000000435914
2.000000000042608
2.00000000004234



### Simultaneous iteration for several eigenvalues

Declare GridFunction with multiple components to store several eigenvectors.


In [6]:
num = 5
u = GridFunction(fes, multidim=num)


Create list of help vectors:


In [7]:
r = u.vec.CreateVector()
Av = u.vec.CreateVector()
Mv = u.vec.CreateVector()

vecs = []
for i in range(2*num):
    vecs.append (u.vec.CreateVector())

for v in u.vecs:
    r.FV().NumPy()[:] = random.rand(fes.ndof)
    v.data = Projector(fes.FreeDofs(), True) * r

# old way
#freedofs = fes.FreeDofs()    
#for v in u.vecs:
#    for i in range(len(u.vec)):
#        v[i] = random() if freedofs[i] else 0



Compute $num$ residuals, and solve small eigenvalue problem on $2num$-dimensional space

In [8]:
asmall = Matrix(2*num, 2*num)
msmall = Matrix(2*num, 2*num)
lams = num * [1]

for i in range(20):
    
    for j in range(num):
        vecs[j].data = u.vecs[j]
        r.data = a.mat * vecs[j] - lams[j] * m.mat * vecs[j]
        vecs[num+j].data = pre.mat * r

    for j in range(2*num):
        Av.data = a.mat * vecs[j]
        Mv.data = m.mat * vecs[j]
        for k in range(2*num):
            asmall[j,k] = InnerProduct(Av, vecs[k])
            msmall[j,k] = InnerProduct(Mv, vecs[k])

    ev,evec = scipy.linalg.eigh(a=asmall, b=msmall)
    lams[:] = ev[0:num]
    print (i, ":", [lam/pi**2 for lam in lams])
    
    for j in range(num):
        u.vecs[j][:] = 0.0
        for k in range(2*num):
            u.vecs[j].data += float(evec[k,j]) * vecs[k]
            
Draw (u)

0 : [11.80364559949426, 70.94139814915454, 83.06716415323355, 92.98351322829299, 100.55685731692716]
1 : [2.0397727982636185, 6.955295610150047, 8.809412950683296, 14.904515815132127, 19.1057980093157]
2 : [2.0013181581679995, 5.1160320175795615, 5.19927277991376, 8.381830230320917, 11.820017249071247]
3 : [2.0001544272216414, 5.00472334510711, 5.016984211794599, 8.065807541736696, 10.317473206738063]
4 : [2.000028597179129, 5.000411724845429, 5.001924124480344, 8.022566822249791, 10.099139861449407]
5 : [2.0000061253403993, 5.000065690167718, 5.000244264763393, 8.0092920570862, 10.035263400540973]
6 : [2.00000134549885, 5.000013334857813, 5.00003438251435, 8.004041421125024, 10.012998569287983]
7 : [2.0000003017031016, 5.000002924698037, 5.0000053801373525, 8.001791874080224, 10.004827477674214]
8 : [2.000000068199269, 5.000000665334123, 5.000000937687826, 8.000798787725156, 10.00179555190104]
9 : [2.0000000155299356, 5.000000156393345, 5.000000182092181, 8.000356887199056, 10.0006685

The multidim-component select in the Visualization dialog box allows to switch between the components of the multidim-GridFunction.